# Import module, data, function

In [1]:
#### import modules
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
import re

# import itertools 
from itertools import chain
from itertools import groupby

In [2]:
# load data set
train_activity = pd.read_csv('../new_input/train_activity_lite.csv')
test_activity = pd.read_csv('../new_input/test_activity_lite.csv')
train_label = pd.read_csv('../new_input/train_label_lite.csv')

# trade
train_trade = pd.read_csv('../new_input/train_trade_lite.csv')
test_trade = pd.read_csv('../new_input/test_trade_lite.csv')

# label map
label_map = {'retained':0,'week':1,'month':2,'2month':3}

In [3]:
# FE된 변수들을 쌓는 변수공간
X_train = train_activity.groupby(by='new_id',sort=False,as_index=False).sum().iloc[:,:1]
X_test = test_activity.groupby(by='new_id',sort=False,as_index=False).sum().iloc[:,:1]
y_train = pd.DataFrame(index = train_label.index)

# label dictionary
label_map = {'retained':0,'week':1,'month':2,'2month':3}

---

# Basic EDA

# Feature engineering

우선 중복 제거

### Basic FE

__1) 주차별 총 거래횟수__

- 주차 별 총 거래횟수 count

해당 id가 있는 전체 row count /2

In [4]:
for i in [1,2,3,4,5,6,7,8]:
    trade_by_week = train_trade.loc[train_trade.trade_week == i,:].iloc[:,3:5].melt().value
    trade_by_week_train = pd.DataFrame(trade_by_week[trade_by_week.apply(lambda x: True if re.match('tr',x)!= None else False)].value_counts())
    trade_by_week_test = pd.DataFrame(trade_by_week[trade_by_week.apply(lambda x: True if re.match('te',x)!= None else False)].value_counts())
    
    X_train['trade_count_' + str(i)] = pd.merge(X_train,trade_by_week_train, left_on='new_id',right_on=trade_by_week_train.index,how='left').fillna(0.).value
    X_test['trade_count_' + str(i)] = pd.merge(X_test,trade_by_week_test, left_on='new_id',right_on=trade_by_week_test.index,how='left').fillna(0.).value
    
    print(i)

1
2
3
4
5
6
7
8


- 주차 별 총 거래횟수 differ

In [5]:
for i in range(1,8):
    X_train['trade_diff_' + str(i)] = X_train.iloc[:,i+1] - X_train.iloc[:,i]
    X_test['trade_diff_' + str(i)] = X_test.iloc[:,i+1] - X_test.iloc[:,i]

- 주차 별 총 거래횟수 variance

In [6]:
X_train['trade_var'] = X_train.iloc[:,1:9].var(axis=1)
X_test['trade_var'] = X_test.iloc[:,1:9].var(axis=1)

__2) 거래물품 비율__

### Network FE

__Weighted Multidigraph__

In [3]:
# 중복 row 제거
train_trade.drop_duplicates(inplace = True)
test_trade.drop_duplicates(inplace = True)
# total_trade 만들기
total_trade = pd.concat((train_trade,test_trade),axis = 0,ignore_index=True)
total_trade.drop_duplicates(inplace = True)

In [22]:
M = nx.MultiDiGraph()

In [ ]:
for item in total_trade.item_type.unique():
    temp = total_trade.loc[total_trade.item_type == item,:].iloc[:,3:].reset_index()
    edge_list = [(temp.source_acc_id[i],t,temp.item_amount[i]) for i,t in enumerate(temp.target_acc_id)]
    M.add_weighted_edges_from(edge_list,relation = item)

In [30]:
for i,e in enumerate(M.edges(data = True)):
    if i > 5 :
        break
    else:
        print(e)

('tr58254', 'ot95753', {'relation': 'grocery', 'weight': -0.0563440169879076})
('tr58254', 'ot95753', {'relation': 'money', 'weight': -0.0476244829174029})
('tr58254', 'ot81514', {'relation': 'grocery', 'weight': -0.0563432959861982})
('tr58254', 'ot81514', {'relation': 'grocery', 'weight': -0.05634400830114})
('tr58254', 'ot81514', {'relation': 'money', 'weight': -0.047636818127371})
('tr58254', 'ot81514', {'relation': 'money', 'weight': -0.0476609499677171})


In [9]:
X_train.to_csv('trade_basic_train.csv',index = False)
X_test.to_csv('trade_basic_test.csv',index = False)